In [33]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import *

from pyspark.sql import Window
from pyspark.sql.functions import rank, col


import pyspark.sql.functions as F 

In [27]:
#Creacion de dataset
df = (spark
  .read                                              
  .option("inferSchema","true")                 
  .option("header","true")                           
  .csv("rating.csv"))

In [28]:
df.count()

20000263

In [20]:
df=df.limit(10000)

In [21]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: string (nullable = true)



In [8]:
df.describe().show()

+-------+------------------+------------------+------------------+-------------------+
|summary|            userId|           movieId|            rating|          timestamp|
+-------+------------------+------------------+------------------+-------------------+
|  count|              1000|              1000|              1000|               1000|
|   mean|             5.071|          1976.229|             3.728|               null|
| stddev|2.9726748553778606|1921.6101144589638|0.9235089977194961|               null|
|    min|                 1|                 1|               0.5|1996-06-05 11:11:21|
|    max|                11|             31696|               5.0|2009-08-25 03:27:37|
+-------+------------------+------------------+------------------+-------------------+



In [10]:
df.describe('timestamp').show()

+-------+-------------------+
|summary|          timestamp|
+-------+-------------------+
|  count|               1000|
|   mean|               null|
| stddev|               null|
|    min|1996-06-05 11:11:21|
|    max|2009-08-25 03:27:37|
+-------+-------------------+



In [16]:
df.filter((df.rating==5.0) & (df.userId==1)).show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|   4993|   5.0|2005-04-02 23:31:22|
|     1|   5952|   5.0|2005-04-02 23:30:19|
|     1|   7153|   5.0|2005-04-02 23:30:33|
|     1|   8507|   5.0|2004-09-10 03:13:47|
+------+-------+------+-------------------+



In [17]:
df.groupBy("rating").count().show()

+------+-----+
|rating|count|
+------+-----+
|   3.5|   80|
|   4.0|  357|
|   3.0|  250|
|   4.5|    9|
|   5.0|  212|
|   2.0|   76|
|   1.0|   13|
|   2.5|    1|
|   0.5|    1|
|   1.5|    1|
+------+-----+



In [23]:
df.sort((df.rating).desc()).show(10)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     2|    589|   5.0|2000-11-21 15:30:58|
|     2|   1544|   5.0|2000-11-21 15:35:43|
|     2|    924|   5.0|2000-11-21 15:36:54|
|     2|    541|   5.0|2000-11-21 15:36:54|
|     2|   1196|   5.0|2000-11-21 15:36:54|
|     1|   7153|   5.0|2005-04-02 23:30:33|
|     2|   1210|   5.0|2000-11-21 15:29:58|
|     2|     62|   5.0|2000-11-21 15:29:58|
|     2|   1214|   5.0|2000-11-21 15:36:54|
|     2|    260|   5.0|2000-11-21 15:36:54|
+------+-------+------+-------------------+
only showing top 10 rows



In [30]:
df.select("userId").distinct().count()

138493

In [29]:
df.select("movieId").distinct().count()

26744

In [32]:
df.withColumn('Half_rating', df.rating/2.0).select('rating','Half_rating').show(10)

+------+-----------+
|rating|Half_rating|
+------+-----------+
|   3.5|       1.75|
|   3.5|       1.75|
|   3.5|       1.75|
|   3.5|       1.75|
|   3.5|       1.75|
|   3.5|       1.75|
|   4.0|        2.0|
|   4.0|        2.0|
|   4.0|        2.0|
|   4.0|        2.0|
+------+-----------+
only showing top 10 rows



In [34]:
df.registerTempTable('df')
sqlContext = SQLContext(spark)

In [35]:
sqlContext.sql('select * from df').show(5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
+------+-------+------+-------------------+
only showing top 5 rows



In [36]:
sqlContext.sql('Select rating, movieId from df').show(5)

+------+-------+
|rating|movieId|
+------+-------+
|   3.5|      2|
|   3.5|     29|
|   3.5|     32|
|   3.5|     47|
|   3.5|     50|
+------+-------+
only showing top 5 rows



In [37]:
sqlContext.sql('SELECT max(timestamp) from df').show(10)

+-------------------+
|     max(timestamp)|
+-------------------+
|2015-03-31 06:40:02|
+-------------------+



In [38]:
sqlContext.sql('select userId, movieId, rating from df where rating = 5.0').show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|   4993|   5.0|
|     1|   5952|   5.0|
|     1|   7153|   5.0|
|     1|   8507|   5.0|
|     2|     62|   5.0|
+------+-------+------+
only showing top 5 rows



In [39]:
sqlContext.sql('SELECT userId, movieId, rating, timestamp from df where rating > 4 and movieId > 1000 ').show(10)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|   1196|   4.5|2005-04-02 23:32:22|
|     1|   1198|   4.5|2005-04-02 23:30:24|
|     1|   4993|   5.0|2005-04-02 23:31:22|
|     1|   5952|   5.0|2005-04-02 23:30:19|
|     1|   7153|   5.0|2005-04-02 23:30:33|
|     1|   8507|   5.0|2004-09-10 03:13:47|
|     1|   8636|   4.5|2005-04-02 23:44:53|
|     2|   1196|   5.0|2000-11-21 15:36:54|
|     2|   1210|   5.0|2000-11-21 15:29:58|
|     2|   1214|   5.0|2000-11-21 15:36:54|
+------+-------+------+-------------------+
only showing top 10 rows

